In [ ]:
# 파이토치 설치하지 않은 경우!
# !pip install torch

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# 데이터 생성 (1000 샘플, 20 특성, 2 클래스)
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# NumPy to PyTorch Tensor
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

## MLP 모델 정의

In [ ]:
class SimpleMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleMLP, self).__init__()
        # fully connected layers
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU() # Activation
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# 모델 인스턴스 생성
input_size = 20
hidden_size = 32
output_size = 2
model = SimpleMLP(input_size, hidden_size, output_size)

print(model)

## 손실 함수 및 옵티마이저 정의

In [ ]:
# 손실 함수: 교차 엔트로피 (분류 문제)
criterion = nn.CrossEntropyLoss()

# 옵티마이저: Adam (경사 하강법 기반)
optimizer = optim.Adam(model.parameters(), lr=0.001)

## 모델 학습 (Training)

In [ ]:
num_epochs = 100

for epoch in range(num_epochs):
    # 순전파
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # 역전파 및 가중치 업데이트
    optimizer.zero_grad() # 기울기 초기화
    loss.backward()       # 역전파
    optimizer.step()      # 가중치 업데이트

    if (epoch+1) % 20 == 0: # 간결하게 출력 주기 조절
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("학습 완료!")

## 모델 평가 (Evaluation)

In [ ]:
# 평가 모드로 변경
model.eval()

# 예측 비활성화 (기울기 계산 X)
with torch.no_grad():
    test_outputs = model(X_test)
    # 가장 높은 확률의 클래스 선택
    _, predicted = torch.max(test_outputs, 1)
    correct = (predicted == y_test).sum().item()
    accuracy = correct / len(y_test)

    print(f'테스트 정확도: {accuracy:.4f}')